In [ ]:
# Étape 1 & 2 - Analyse exploratoire + Score métier (projet Home Credit)
# Objectif : analyser les données et créer une fonction métier pour évaluer un modèle de scoring crédit

# ----------------------
# IMPORT DES LIBRAIRIES
# ----------------------
import pandas as pd  # gestion des données
import numpy as np  # calculs numériques
import matplotlib.pyplot as plt  # visualisation simple
import seaborn as sns  # visualisation plus avancée
import missingno as msno  # visualisation des valeurs manquantes
import warnings  # pour cacher les avertissements
from sklearn.model_selection import train_test_split  # découper les données
from sklearn.linear_model import LogisticRegression  # modèle simple et interprétable
from sklearn.metrics import confusion_matrix  # évaluer les erreurs de prédiction

warnings.filterwarnings('ignore')  # éviter d'afficher les messages de warning inutiles

# -------------------------------------
# CHARGEMENT DES DONNÉES
# -------------------------------------
file_path = "//content/drive/MyDrive/Master AI Engineer/Colab Notebooks/Projet 4/application_train.csv"

# On essaie d'abord avec une virgule, sinon on essaye avec point-virgule
try:
    df = pd.read_csv(file_path, sep=",", encoding="utf-8", low_memory=False)
    if df.shape[1] == 1:
        raise ValueError("Colonnes mal séparées")
except:
    df = pd.read_csv(file_path, sep=";", encoding="utf-8", low_memory=False)

# -------------------------------------
# ANALYSE EXPLORATOIRE
# -------------------------------------
print("Aperçu des données :")
print(df.head())
print("\nDimensions :", df.shape)
print("\nTypes de données :")
print(df.dtypes.value_counts())
print("\nInfos détaillées :")
df.info()

# Valeurs manquantes
missing = df.isnull().mean().sort_values(ascending=False) * 100
missing = missing[missing > 0]
if not missing.empty:
    plt.figure(figsize=(10, 12))
    missing.plot(kind="barh", title="Pourcentage de valeurs manquantes par colonne")
    plt.xlabel("Pourcentage de valeurs manquantes")
    plt.tight_layout()
    plt.show()
else:
    print("Aucune valeur manquante détectée.")

# Statistiques globales
print("\nStatistiques générales :")
print(df.describe().T)

# Distribution de la cible (TARGET)
if 'TARGET' in df.columns:
    plt.figure(figsize=(6, 4))
    df['TARGET'].value_counts(normalize=True).plot(kind='bar', color=['skyblue', 'salmon'])
    plt.title("Répartition de la variable cible (TARGET)")
    plt.xlabel("0 = Bon client, 1 = Mauvais client")
    plt.ylabel("Proportion")
    plt.show()

# -------------------------------------
# FEATURE ENGINEERING - NOUVELLES VARIABLES
# -------------------------------------
print("\nCréation de nouvelles variables :")
if {'AMT_CREDIT', 'AMT_INCOME_TOTAL'}.issubset(df.columns):
    df['CREDIT_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    print("CREDIT_INCOME_RATIO ajouté.")

if {'AMT_ANNUITY', 'AMT_INCOME_TOTAL'}.issubset(df.columns):
    df['ANNUITY_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    print("ANNUITY_INCOME_RATIO ajouté.")

if {'DAYS_EMPLOYED', 'DAYS_BIRTH'}.issubset(df.columns):
    df['EMPLOYED_AGE_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    print("EMPLOYED_AGE_RATIO ajouté.")

# -------------------------------------
# CORRÉLATION AVEC LA CIBLE
# -------------------------------------
if 'TARGET' in df.columns:
    correlations = df.corr(numeric_only=True)['TARGET'].sort_values(ascending=False)
    print("\nTop 10 variables les plus corrélées positivement avec le défaut :")
    print(correlations.head(10))
    print("\nTop 10 variables les plus corrélées négativement avec le défaut :")
    print(correlations.tail(10))

# -------------------------------------
# EXPLORATION DES VARIABLES CATÉGORIELLES
# -------------------------------------
print("\nExploration des variables catégorielles :")
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    print(f"\n{col} :")
    print(df[col].value_counts(dropna=False).head())

# -------------------------------------
# ÉTAPE 2 : SCORING MÉTIER ET SEUIL OPTIMAL
# -------------------------------------
# Nettoyage simple : suppression des NaN, sélection des colonnes numériques uniquement
# (pour simplifier, on ne conserve ici que les colonnes sans NaN pour ce premier test)

df_model = df.dropna()
features = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'CREDIT_INCOME_RATIO', 'ANNUITY_INCOME_RATIO', 'EMPLOYED_AGE_RATIO']
X = df_model[features]
y = df_model['TARGET']

# Découpage des données en 80% train / 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Modèle de régression logistique (modèle simple et interprétable)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Prédiction des probabilités pour le test
y_proba = model.predict_proba(X_test)[:, 1]  # on récupère les probabilités d'appartenir à la classe 1

# ------------------
# Fonction de coût métier
# ------------------
def cout_metier(y_true, y_pred, cost_fn=10, cost_fp=1):
    """Calcule le coût total en fonction du nombre de faux négatifs et faux positifs"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return cost_fn * fn + cost_fp * fp

# ------------------
# Fonction pour trouver le meilleur seuil
# ------------------
def trouver_seuil_optimal(y_true, y_proba, seuils=np.arange(0.1, 0.9, 0.01), cost_fn=10, cost_fp=1):
    meilleur_seuil = None
    meilleur_cout = float('inf')

    for seuil in seuils:
        y_pred = (y_proba >= seuil).astype(int)
        cout = cout_metier(y_true, y_pred, cost_fn, cost_fp)
        if cout < meilleur_cout:
            meilleur_cout = cout
            meilleur_seuil = seuil

    return {"seuil": meilleur_seuil, "coût": meilleur_cout}

# Application sur les résultats
resultat = trouver_seuil_optimal(y_test, y_proba)
print(f"\n✅ Seuil optimal = {resultat['seuil']:.2f}, coût total = {resultat['coût']}")

# Affichage graphique du coût selon le seuil choisi
seuils = np.arange(0.1, 0.9, 0.01)
costs = [cout_metier(y_test, (y_proba >= s).astype(int)) for s in seuils]

plt.figure(figsize=(8, 4))
plt.plot(seuils, costs, label="Coût métier")
plt.axvline(resultat['seuil'], color='red', linestyle='--', label='Seuil optimal')
plt.xlabel("Seuil de classification")
plt.ylabel("Coût total (pondéré)")
plt.title("Optimisation du seuil de décision")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ✅ Fin de l'étape 2 : modèle interprétable + seuil optimisé selon le coût métier



# ---------------------------------------------------------------
# ÉTAPE 3 – Benchmark de modèles : Régression Logistique vs Random Forest
# Objectif : tester plusieurs modèles et les comparer en tenant compte :
# - du déséquilibre entre bons et mauvais clients
# - du coût métier (FN plus grave que FP)
# ---------------------------------------------------------------

# Import de bibliothèques supplémentaires pour cette étape
from sklearn.ensemble import RandomForestClassifier  # un modèle plus complexe que la régression
from sklearn.model_selection import cross_val_score  # pour faire de la validation croisée
from sklearn.pipeline import Pipeline  # permet d’enchaîner plusieurs étapes dans un même objet
from sklearn.preprocessing import StandardScaler  # pour normaliser les données
from sklearn.metrics import roc_auc_score, roc_curve  # métriques de performance
from imblearn.over_sampling import SMOTE  # technique pour équilibrer les classes
from imblearn.pipeline import Pipeline as ImbPipeline  # pipeline compatible avec SMOTE

# On refait un découpage 80/20 pour bien séparer l'entraînement et le test
# stratify permet de garder le même % de mauvais clients dans chaque groupe
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Liste des modèles qu'on veut comparer
# Un modèle simple (logistique) et un plus puissant (forêt aléatoire)
modeles = [
    ("Régression Logistique", LogisticRegression(max_iter=1000)),
    ("Random Forest", RandomForestClassifier(n_estimators=100, random_state=42))
]

# On prépare une liste pour stocker les résultats de chaque modèle
resultats = []

# On teste chaque modèle un par un
for nom_modele, modele in modeles:
    # On crée un "pipeline" : c’est un enchaînement d’étapes automatiques
    # 1. StandardScaler : normalise les données (utile pour la régression logistique)
    # 2. SMOTE : augmente artificiellement les exemples de mauvais clients (classe minoritaire)
    # 3. Le modèle choisi
    pipeline = ImbPipeline(steps=[
        ('scaler', StandardScaler()),
        ('smote', SMOTE(random_state=42)),
        ('model', modele)
    ])

    # On évalue la performance du modèle via une validation croisée (5 parties du jeu d'entraînement)
    # Cela évite de tirer des conclusions trop hâtives sur un seul découpage
    scores_auc = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=5)
    moyenne_auc = np.mean(scores_auc)  # moyenne des AUC obtenues

    # On entraîne le pipeline sur tout le jeu d'entraînement
    pipeline.fit(X_train, y_train)

    # On prédit les probabilités pour chaque client du jeu de test
    y_proba = pipeline.predict_proba(X_test)[:, 1]  # on garde la proba d’être un mauvais client (classe 1)

    # On cherche le seuil de proba qui minimise notre score métier (FN = 10x FP)
    resultat_seuil = trouver_seuil_optimal(y_test, y_proba)

    # On transforme les proba en prédiction 0 ou 1 avec le bon seuil
    y_pred = (y_proba >= resultat_seuil['seuil']).astype(int)

    # On mesure la qualité globale via l’AUC (plus c’est proche de 1, mieux c’est)
    auc_test = roc_auc_score(y_test, y_proba)

    # On enregistre les résultats
    resultats.append({
        "Modèle": nom_modele,
        "AUC Test": round(auc_test, 3),
        "Seuil Métier": round(resultat_seuil['seuil'], 2),
        "Coût Métier": resultat_seuil['coût']
    })

# On affiche un tableau récapitulatif des performances
resultats_df = pd.DataFrame(resultats)
print("\n📊 Résumé des performances par modèle :")
print(resultats_df)

# BONUS : on trace la courbe ROC pour le meilleur modèle (ici Random Forest)
# ROC = Receiver Operating Characteristic
# C’est une courbe qui montre le compromis entre :
# - vrai positifs (clients mal classés détectés)
# - faux positifs (bons clients à qui on refuse à tort)
# 3. On réentraîne un modèle Random Forest sur tout l'ensemble d'entraînement pour l'analyse SHAP
# On enlève StandardScaler ici car RandomForest n'en a pas besoin
best_model = RandomForestClassifier(n_estimators=100, random_state=42)
best_pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),  # on garde SMOTE pour équilibrer les classes
    ('model', best_model)
])
best_pipeline.fit(X_train, y_train)

y_proba_best = best_pipeline.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_proba_best)  # fpr = faux positifs, tpr = vrais positifs

plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label="ROC - Random Forest")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # diagonale : tirage au sort
plt.xlabel("Taux de faux positifs")
plt.ylabel("Taux de vrais positifs")
plt.title("Courbe ROC - Random Forest")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ✅ Étape 3 terminée : on a comparé 2 modèles, géré le déséquilibre, mesuré l’AUC et le score métier

# ---------------------------------------------------------------
# ÉTAPE 4 – Interprétation du modèle avec SHAP (importance globale et locale)
# Objectif : expliquer comment le modèle prend ses décisions
# ---------------------------------------------------------------

# 1. On importe la librairie SHAP (permet d’interpréter comment le modèle raisonne)
import shap

# 2. On réentraîne un modèle Random Forest tout seul, sans pipeline ni SMOTE
# Pourquoi ? Parce que SHAP a besoin du modèle brut (pas d’un pipeline) et des données originales
modele_shap = RandomForestClassifier(n_estimators=100, random_state=42)
modele_shap.fit(X_train, y_train)  # entraînement sur les données d'entraînement d'origine

# 3. On crée un "explainer" SHAP adapté aux modèles d’arbres (comme RandomForest)
explainer = shap.TreeExplainer(modele_shap)

# -------------------------------
# échantillonnage pour accélérer le calcul SHAP
# On sélectionne 200 clients du jeu de test, ça suffit pour l’analyse globale
# Cela évite que SHAP prenne plusieurs minutes à tout calculer
X_test_sample_df = X_test.sample(n=200, random_state=42).copy()  # on prend un sous-échantillon du test

# 4. On calcule les valeurs SHAP avec la nouvelle API
shap_values = explainer(X_test_sample_df)

# 5. Affichage global des variables les plus influentes
shap.summary_plot(shap_values.values, X_test_sample_df, plot_type="bar")

#Interpretation locale


# Besoin d'aide ici @Gred

# ---------------------------------------------------------------
# ÉTAPE 5 – DummyClassifier + Optimisation Random Forest avec GridSearchCV
# Objectif : répondre aux attentes restantes du mentor :
# - Ajouter un modèle de référence (très basique) appelé "DummyClassifier"
# - Utiliser GridSearchCV pour tester plusieurs combinaisons de paramètres
# - Optimiser le modèle en fonction de notre score métier
# - Mesurer aussi l'AUC (qualité globale) et le temps d'exécution
# ---------------------------------------------------------------

from sklearn.dummy import DummyClassifier  # modèle très simple pour établir une baseline
from sklearn.metrics import make_scorer  # pour créer une métrique personnalisée
from sklearn.model_selection import GridSearchCV  # outil pour tester plusieurs réglages automatiques
import time  # pour chronométrer l'entraînement

# ➜ On commence par un modèle de base (Dummy) qui prédit toujours la majorité
dummy = DummyClassifier(strategy="most_frequent")  # toujours prédire 0
dummy.fit(X_train, y_train)
y_pred_dummy = dummy.predict(X_test)
cout_dummy = cout_metier(y_test, y_pred_dummy)
print(f"\n📌 Coût métier DummyClassifier (baseline naïve) : {cout_dummy}")

# ➜ On va maintenant définir une fonction personnalisée pour que GridSearch comprenne notre score métier
def scoring_metier(y_true, y_proba):
    y_pred = (y_proba >= 0.5).astype(int)  # on garde un seuil fixe ici (comme demandé)
    return -cout_metier(y_true, y_pred)  # on inverse car GridSearch cherche à *maximiser* le score

# make_scorer transforme notre fonction en objet compatible avec sklearn
scorer = make_scorer(scoring_metier, needs_proba=True)

# ➜ On définit les paramètres qu'on veut tester pour la Random Forest
param_grid = {
    'model__n_estimators': [50, 100],  # nombre d’arbres dans la forêt
    'model__max_depth': [5, 10, None]  # profondeur maximale des arbres
}

# ➜ Pipeline avec normalisation + SMOTE + RandomForest (comme à l’étape 3)
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('model', RandomForestClassifier(random_state=42))
])

# ➜ On lance GridSearchCV en demandant :
# - d’optimiser à la fois notre score métier et l’AUC
# - de choisir le modèle avec le *meilleur score métier*
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring={'metier': scorer, 'auc': 'roc_auc'},
    refit='metier',
    cv=5,
    return_train_score=True
)

# ➜ On lance l'entraînement complet avec chronomètre
start = time.time()
grid.fit(X_train, y_train)
end = time.time()

# ➜ Affichage des meilleurs paramètres et scores associés
print("\n✅ Meilleurs paramètres trouvés par GridSearch :")
print(grid.best_params_)
print(f"Score métier moyen (sur le train cross-validé) : {-grid.best_score_:.0f}")
print(f"Temps total d'entraînement : {end - start:.1f} secondes")

# ➜ Vérification du modèle obtenu sur le jeu de test
y_proba_grid = grid.predict_proba(X_test)[:, 1]
resultat_grid = trouver_seuil_optimal(y_test, y_proba_grid)

print(f"\n🔍 Vérification sur le test : seuil optimal = {resultat_grid['seuil']:.2f}")
print(f"Coût métier sur le test = {resultat_grid['coût']}")



# ---------------------------------------------------------------
# BONUS – Interprétation locale avec LIME (alternative à SHAP)
# Objectif : donner une explication simple et visuelle d’une prédiction
# ---------------------------------------------------------------

# 1. On installe la librairie LIME (si elle n’est pas déjà installée)
# À exécuter une seule fois dans l’environnement Colab ou Jupyter
# !pip install lime

from lime.lime_tabular import LimeTabularExplainer  # outil d’explication locale

# 2. On entraîne un modèle simple (sans pipeline) car LIME ne comprend pas les pipelines non plus
modele_lime = RandomForestClassifier(n_estimators=100, random_state=42)
modele_lime.fit(X_train, y_train)

# 3. On crée un "explainer" LIME
# ➜ Il apprend à expliquer les décisions à partir des données d’entraînement
explainer_lime = LimeTabularExplainer(
    training_data=X_train.values,         # données d’entraînement
    feature_names=X_train.columns,        # noms des colonnes pour l'affichage
    class_names=['Bon client', 'Mauvais client'],  # nom des classes
    mode='classification',                # on fait de la classification (pas de la régression)
    verbose=True,                         # pour avoir des infos dans les logs
    random_state=42
)

# 4. On choisit un client à expliquer dans le jeu de test
# ➜ Tu peux changer l’index si tu veux un autre client
index_client = 0
client_data = X_test.iloc[index_client].values.reshape(1, -1)  # données du client

# 5. On génère l’explication LIME
explication = explainer_lime.explain_instance(
    data_row=X_test.iloc[index_client].values,   # données du client
    predict_fn=modele_lime.predict_proba,        # fonction de prédiction du modèle
    num_features=6                               # on limite à 6 variables les plus influentes
)

# 6. On affiche l’explication
explication.show_in_notebook(show_table=True)

# ➜ Ce graphique est très pédagogique :
# Il montre, pour un seul client, quelles sont les variables qui ont poussé le modèle à prédire "mauvais" ou "bon".
# Très utile à montrer à quelqu’un qui ne connaît pas le machine learning.